In [3]:
# check gpu
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
# load data and split into train and test sets
from load_data import sim_arr
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(sim_arr, test_size=0.2, random_state=42)

In [5]:
from autoencoder import Autoencoder
from hyper_optim import Hyper

# hyperparameter optimization
hyper = Hyper(Autoencoder, 3, train_set, test_set)
study = hyper.optimize(direction="minimize",
                       study_name="autoencoder",
                       storage="autoencoder.db",
                       n_trials=1)

# Get the best hyperparameters
best_params = study.best_params
for key, value in best_params.items():
    if key == 'activation':
        print(key, ":", value.__name__)
    else:
        print(key, ":", value)
print("Best hyperparameters:", best_params)
print("Best value:", study.best_value)

[I 2024-04-22 23:15:40,646] Using an existing study with name 'autoencoder' instead of creating a new one.


  0%|          | 0/1 [00:00<?, ?it/s]

/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2024-04-22 23:15:40.811896: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 23:15:40.812460: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 23:15:40.812947: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA

[W 2024-04-22 23:15:40,997] Trial 3 failed with parameters: {'num_layers': 2, 'filters_0': 4, 'kernel_size0': 9, 'kernel_size1': 22, 'strides_0': 2, 'strides_1': 2, 'activation_0': 'selu', 'activation_1': 'relu', 'padding_0': 'valid', 'padding_1': 'valid', 'initializer_0': 'he_uniform', 'initializer_1': 'glorot_normal'} because of the following error: ValueError('Computed output size would be negative. Received `inputs shape=(None, 1, 1, 4)`, `kernel shape=(22, 22, 4, 3)`, `dilation_rate=[1 1]`.').
Traceback (most recent call last):
  File "/home/tux/hyper-param-optim-4-vector-field-clustering/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/home/tux/hyper-param-optim-4-vector-field-clustering/hyper_optim.py", line 57, in objective
    model = self.model(*self.search_space(trial),
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tux/hyper-param-optim-4-vecto

ValueError: Computed output size would be negative. Received `inputs shape=(None, 1, 1, 4)`, `kernel shape=(22, 22, 4, 3)`, `dilation_rate=[1 1]`.

In [ ]:
# train the model with the best hyperparameters
hyperparams = Hyper.search_space(study.best_trial)
model = Autoencoder(*hyperparams, output_dim=3)

model.fit(train_set, test_set, epochs=10, batch_size=32)
model.evaluate(test_set, test_set)

In [ ]:
# summarize the model
model.get_layer('encoder').summary()
model.get_layer('decoder').summary()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i = np.random.randint(0, len(test_set))
in_sim = test_set[i: i + 1]
out_sim = model(in_sim)
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(in_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")
ax[1].imshow(out_sim[0, ..., 2], vmin=-1, vmax=1, cmap="RdBu")

In [ ]:
from tensorflow.keras.layers import Flatten as Flatten

feature_arr = model.feature_extractor(sim_arr)

In [ ]:
plt.scatter(feature_arr[:, 0],
            feature_arr[:, 1],
            feature_arr[:, 2])